## 창원시 교차점 사이 모든 길 DataFrame으로 저장 length의 단위는 m임

In [3]:
import os

os.chdir('D:\\창원시공모전') # 작업 디렉토리 변경
os.getcwd() # 현재 작업 디렉토리

import geopandas as gpd # GeoPandas(지오판다스)
from geopandas import GeoDataFrame
import glob # glob(글로브)
import rasterio
import pandas as pd
import pandas as pd
import osmnx as ox 
import numpy as np 
import folium 
import matplotlib.pyplot as plt 
import networkx as nx
from shapely import wkt
from tqdm import tqdm
from shapely.geometry import Point, LineString, Polygon

import warnings
warnings.simplefilter("ignore")

In [2]:
# 창원시엣지 (osmnx라이브러리를 이용하여 뽑은 창원시 엣지 노드)
changwon_edge = pd.read_csv('ChangwonEdge_1111.csv')
changwon_edge

,Unnamed: 0,u,v,key,osmid,name,highway,oneway,length,geometry,u_original,v_original,slope,count,TAAS,sz
0,0,0,1,0,383421263,회원동24길,residential,False,80.825582,LINESTRING (128.57720125002797 35.220253350013...,356102207,3866854856,0,0,0,0
1,1,0,2,0,907575424,회원동북로,secondary,False,169.309167,LINESTRING (128.57720125002797 35.220253350013...,356102207,2910221305,0,0,0,0
2,2,0,4005,0,907575424,회원동북로,secondary,False,147.320611,LINESTRING (128.57720125002797 35.220253350013...,4665552566,3866854853,0,0,0,0
3,3,0,2471,0,"[472344889, 208055347]","['회원동34길', '산호서8길']",residential,False,361.812547,LINESTRING (128.57720125002797 35.220253350013...,4665552566,2183525893,0,0,0,0
4,4,1,0,0,383421263,회원동24길,residential,False,80.825582,"LINESTRING (128.5766063 35.2197931, 128.577144...",3866854856,356102207,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34829,34829,10306,10305,1,1060056482,NaN,service,False,159.960000,"LINESTRING (128.5966053 35.2442068, 128.596593...",9739424082,9739424075,0,0,0,0
34830,34830,10306,10307,0,1060056480,NaN,service,False,83.952000,"LINESTRING (128.5966053 35.2442068, 128.596804...",9739424082,9739424083,0,0,0,0
34831,34831,10314,10312,0,1060164460,NaN,service,False,54.469000,"LINESTRING (128.6116203 35.2440486, 128.611586...",9740451680,9740451702,0,0,0,0
34832,34832,10314,10313,0,1060164461,NaN,service,False,44.872000,"LINESTRING (128.6116203 35.2440486, 128.611601...",9740451680,9740451671,0,0,0,0


In [4]:
# 필요한 컬럼만 선택
new_changwon = changwon_edge[['u','v','name','length','geometry']]
new_changwon

,u,v,name,length,geometry
0,0,1,회원동24길,80.825582,LINESTRING (128.57720125002797 35.220253350013...
1,0,2,회원동북로,169.309167,LINESTRING (128.57720125002797 35.220253350013...
2,0,4005,회원동북로,147.320611,LINESTRING (128.57720125002797 35.220253350013...
3,0,2471,"['회원동34길', '산호서8길']",361.812547,LINESTRING (128.57720125002797 35.220253350013...
4,1,0,회원동24길,80.825582,"LINESTRING (128.5766063 35.2197931, 128.577144..."
...,...,...,...,...,...
34829,10306,10305,NaN,159.960000,"LINESTRING (128.5966053 35.2442068, 128.596593..."
34830,10306,10307,NaN,83.952000,"LINESTRING (128.5966053 35.2442068, 128.596804..."
34831,10314,10312,NaN,54.469000,"LINESTRING (128.6116203 35.2440486, 128.611586..."
34832,10314,10313,NaN,44.872000,"LINESTRING (128.6116203 35.2440486, 128.611601..."


In [5]:
# geometry 정의 
new_changwon['geometry'] = new_changwon['geometry'].apply(wkt.loads)

In [7]:
# 좌표계 정의후 파일을 저장함 (엣지마다 포인트 생성하는 작업은 QGIS에서 전처리 진행함)
gdf = GeoDataFrame(new_changwon, crs="EPSG:4326", geometry='geometry')

# GeoDataFrame을 Shapefile로 저장
gdf.crs= "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
gdf.to_file('new_changwon_linestring.shp', driver='ESRI Shapefile')

### 프로젝트 폴리곤 범위

In [9]:
# 창원시 전체지역에서 마산합포구, 마산해원구 범위만 프로젝트 범위로 사용
gdf_polygon = gpd.GeoDataFrame.from_file("폴리곤/polygon.shp")

In [10]:
# 필요없는 컬럼 삭제
gdf_polygon.drop(['CENTER_CD', 'LOCATION_Y', 'LOCATION_D', 'ADDRESS_JI',
       'LINK_ID', 'VIEW_YN', 'VIEW_CNT', 'REPORT_DAT', 'ADDRESS_NE', 'L_CD',
       'M_CD', 'S_CD', 'TYPE_TOP_C', 'TYPE_MID_C', 'TYPE_BOT_C', 'INCIDENT_R',
       'REPORT_ID', 'LOCATION_T', 'REG_DATE', 'ADDR_CD', 'LOCATION_X',
       'LOCATION_1', 'DATA_DESC', 'f_nm', 'f_type', 'flag'], axis=1, inplace=True)
gdf_polygon

,SAFEDATA_I,geometry
0,합포구,"POLYGON ((128.55055 35.18470, 128.55801 35.201..."


### 새로운 창원시 엣지 노드 기준으로 추출된 포인트 좌표 전처리

#### QGIS에서 전처리한 엣지 마다 포인트 생성한 파일 -> 엣지 마다 포인트 5개씩 일정한 구간으로 생성 이는 경사각도 데이터를 더 세밀하게 맵핑시키기 위함

In [11]:
point1 = gpd.GeoDataFrame.from_file("폴리곤/point1.shp")
point2 = gpd.GeoDataFrame.from_file("폴리곤/point2.shp")
point3 = gpd.GeoDataFrame.from_file("폴리곤/new_point.shp")

In [12]:
# 포인트 데이터 합쳐줌
a = point2.append(point1, ignore_index = True)
df_point = point3.append(a, ignore_index = True)
df_point

,u,v,name,length,vertex_ind,vertex_par,vertex_p_1,distance,angle,geometry,fid,source,target,u_original,v_original
0,0.0,1.0,???24?,80.825582,0.0,0.0,0.0,0.000000,305.069438,POINT (128.57720 35.22025),NaN,NaN,NaN,NaN,NaN
1,0.0,1.0,???24?,80.825582,1.0,0.0,1.0,0.000070,266.071793,POINT (128.57714 35.22029),NaN,NaN,NaN,NaN,NaN
2,0.0,1.0,???24?,80.825582,2.0,0.0,2.0,0.000804,227.074148,POINT (128.57661 35.21979),NaN,NaN,NaN,NaN,NaN
3,0.0,2.0,?????,169.309167,0.0,0.0,0.0,0.000000,305.069438,POINT (128.57720 35.22025),NaN,NaN,NaN,NaN,NaN
4,0.0,2.0,?????,169.309167,1.0,0.0,1.0,0.000070,304.096893,POINT (128.57714 35.22029),NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235101,NaN,NaN,None,108.706000,NaN,NaN,NaN,NaN,NaN,POINT (128.59633 35.24412),34760.0,10295.0,10294.0,9.739424e+09,9.739424e+09
235102,NaN,NaN,None,159.960000,NaN,NaN,NaN,NaN,NaN,POINT (128.59632 35.24439),34761.0,10295.0,10294.0,9.739424e+09,9.739424e+09
235103,NaN,NaN,None,83.952000,NaN,NaN,NaN,NaN,NaN,POINT (128.59680 35.24427),34762.0,10295.0,10296.0,9.739424e+09,9.739424e+09
235104,NaN,NaN,None,54.469000,NaN,NaN,NaN,NaN,NaN,POINT (128.61157 35.24393),34763.0,10303.0,10301.0,9.740452e+09,9.740452e+09


In [13]:
# 폴리곤 내부에 있는지 확인하기 위한 빈 컬럼 생성
df_point['inside'] = 0

In [14]:
# geometry에서 위도, 경도 컬럼 생성함
df_point['lon'] = df_point['geometry'].x
df_point['lat'] = df_point['geometry'].y

In [15]:
Polygon = gdf_polygon
# 위에서 추출한 포인트 데이터 geometry컬럼 기준으로 폴리곤 내부에 포인트가 있는지 확인
for i in tqdm(range(len(Polygon))):
    df_point['inside'] = df_point.apply(lambda x: Polygon.contains(Point(x['lon'], x['lat'])), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:30<00:00, 30.78s/it]


In [16]:
df_point['inside'].value_counts()

False    210065
True      25041
Name: inside, dtype: int64

In [17]:
# 범위 안에 들어가는 포인트들만 저장
df_point = df_point[(df_point['inside'] == True)]

### 범위 안에 있는 포인트들의 경사도 값 출력

In [18]:
df_point

,u,v,name,length,vertex_ind,vertex_par,vertex_p_1,distance,angle,geometry,fid,source,target,u_original,v_original,inside,lon,lat
0,0.0,1.0,???24?,80.825582,0.0,0.0,0.0,0.000000,305.069438,POINT (128.57720 35.22025),NaN,NaN,NaN,NaN,NaN,True,128.577201,35.220253
1,0.0,1.0,???24?,80.825582,1.0,0.0,1.0,0.000070,266.071793,POINT (128.57714 35.22029),NaN,NaN,NaN,NaN,NaN,True,128.577144,35.220293
2,0.0,1.0,???24?,80.825582,2.0,0.0,2.0,0.000804,227.074148,POINT (128.57661 35.21979),NaN,NaN,NaN,NaN,NaN,True,128.576606,35.219793
3,0.0,2.0,?????,169.309167,0.0,0.0,0.0,0.000000,305.069438,POINT (128.57720 35.22025),NaN,NaN,NaN,NaN,NaN,True,128.577201,35.220253
4,0.0,2.0,?????,169.309167,1.0,0.0,1.0,0.000070,304.096893,POINT (128.57714 35.22029),NaN,NaN,NaN,NaN,NaN,True,128.577144,35.220293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233939,NaN,NaN,None,68.008132,NaN,NaN,NaN,NaN,NaN,POINT (128.55726 35.19888),33598.0,9693.0,9692.0,9.134749e+09,9.134749e+09,True,128.557264,35.198884
233940,NaN,NaN,None,128.953748,NaN,NaN,NaN,NaN,NaN,POINT (128.55732 35.19841),33599.0,9693.0,9693.0,9.134749e+09,9.134749e+09,True,128.557316,35.198407
233941,NaN,NaN,None,128.953748,NaN,NaN,NaN,NaN,NaN,POINT (128.55708 35.19859),33600.0,9693.0,9693.0,9.134749e+09,9.134749e+09,True,128.557082,35.198592
234132,NaN,NaN,None,52.845000,NaN,NaN,NaN,NaN,NaN,POINT (128.55551 35.17990),33791.0,9860.0,9859.0,9.406700e+09,9.406700e+09,True,128.555510,35.179905


In [19]:
# 포인트 좌표 리스트
coord_list = [(x,y) for x,y in zip(df_point['lon'], df_point['lat'])]
print(coord_list[0:5])
# x, y좌표 경도, 위도를 zip함수로 묶어줌

[(128.57720125002797, 35.22025335001377), (128.5771442, 35.2202934), (128.5766063, 35.2197931), (128.57720125002797, 35.22025335001377), (128.5771442, 35.2202934)]


In [20]:
# 수치지형도에서 등고선 데이터 가져와서 고도 데이터 변환후 경사도를 구했음(QGIS에서 전처리 진행)
# 파일 포맷은 GeoTiFF임
# 래스터 특징

# glob.glob 함수를 통해 특정 디렉토리(폴더)로 부터 디렉토리 및 파일 목록을 가져와서 리스트를 보면 정렬되지 않는 경우,
raster_features = sorted(glob.glob(
    '경사도/전체경사도crs.tif'))
print(raster_features)

['경사도/전체경사도crs.tif']


In [21]:
from pathlib import Path # pathlib

# 각 위도 경도 위치를 래스터 이미지 하나씩 가지고와서 맵핑시켜줌 
# 예를 들어 특정 위도,경도에 경사도 데이터를 하나씩 매칭하는 형태
for file in raster_features:
    src = rasterio.open(file) # 파일 읽기
    df_point['경사도'] = [x for x in src.sample(coord_list)] # 포인트 샘플링
    df_point['경사도'] = df_point['경사도'].astype('float64')

In [22]:
df_point

,u,v,name,length,vertex_ind,vertex_par,vertex_p_1,distance,angle,geometry,fid,source,target,u_original,v_original,inside,lon,lat,경사도
0,0.0,1.0,???24?,80.825582,0.0,0.0,0.0,0.000000,305.069438,POINT (128.57720 35.22025),NaN,NaN,NaN,NaN,NaN,True,128.577201,35.220253,4.458383
1,0.0,1.0,???24?,80.825582,1.0,0.0,1.0,0.000070,266.071793,POINT (128.57714 35.22029),NaN,NaN,NaN,NaN,NaN,True,128.577144,35.220293,6.524668
2,0.0,1.0,???24?,80.825582,2.0,0.0,2.0,0.000804,227.074148,POINT (128.57661 35.21979),NaN,NaN,NaN,NaN,NaN,True,128.576606,35.219793,1.595524
3,0.0,2.0,?????,169.309167,0.0,0.0,0.0,0.000000,305.069438,POINT (128.57720 35.22025),NaN,NaN,NaN,NaN,NaN,True,128.577201,35.220253,4.458383
4,0.0,2.0,?????,169.309167,1.0,0.0,1.0,0.000070,304.096893,POINT (128.57714 35.22029),NaN,NaN,NaN,NaN,NaN,True,128.577144,35.220293,6.524668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233939,NaN,NaN,None,68.008132,NaN,NaN,NaN,NaN,NaN,POINT (128.55726 35.19888),33598.0,9693.0,9692.0,9.134749e+09,9.134749e+09,True,128.557264,35.198884,8.681136
233940,NaN,NaN,None,128.953748,NaN,NaN,NaN,NaN,NaN,POINT (128.55732 35.19841),33599.0,9693.0,9693.0,9.134749e+09,9.134749e+09,True,128.557316,35.198407,21.686089
233941,NaN,NaN,None,128.953748,NaN,NaN,NaN,NaN,NaN,POINT (128.55708 35.19859),33600.0,9693.0,9693.0,9.134749e+09,9.134749e+09,True,128.557082,35.198592,7.144462
234132,NaN,NaN,None,52.845000,NaN,NaN,NaN,NaN,NaN,POINT (128.55551 35.17990),33791.0,9860.0,9859.0,9.406700e+09,9.406700e+09,True,128.555510,35.179905,6.715435


In [23]:
df_point1 = df_point[['geometry','경사도']]

In [24]:
df_point1.to_csv('폴리곤내부포인트.csv')

## 창원시 노드 데이터로 TAAS, SAFETY_ZONE 전처리

In [84]:
# 창원시노드
changwon_node = pd.read_csv('ChangwonNode_1110.csv')
changwon_node

,osmid,osmid_original,x,y,street_count,lon,lat,highway,ref,geometry
0,0,"[356102207, 4665552566]",128.577201,35.220253,0,NaN,NaN,NaN,NaN,POINT (128.57720125002797 35.22025335001377)
1,1,3866854856,128.576606,35.219793,4,128.576606,35.219793,NaN,NaN,POINT (128.5766063 35.2197931)
2,2,2910221305,128.575747,35.221205,3,128.575747,35.221205,NaN,NaN,POINT (128.5757468 35.2212052)
3,4005,3866854853,128.578431,35.219390,4,128.578431,35.219390,NaN,NaN,POINT (128.5784315 35.21938999999999)
4,2471,2183525893,128.580741,35.219552,3,128.580741,35.219552,NaN,NaN,POINT (128.5807414 35.219552199999995)
...,...,...,...,...,...,...,...,...,...,...
10314,10304,9740469241,128.595765,35.241808,3,128.595765,35.241808,NaN,NaN,POINT (128.5957645 35.2418085)
10315,10305,9739424075,128.595491,35.243850,3,128.595491,35.243850,NaN,NaN,POINT (128.5954908 35.2438501)
10316,10306,9739424082,128.596605,35.244207,3,128.596605,35.244207,NaN,NaN,POINT (128.5966053 35.2442068)
10317,10314,9740451680,128.611620,35.244049,3,128.611620,35.244049,NaN,NaN,POINT (128.6116203 35.2440486)


In [85]:
# 폴리곤 내부에 있는지 확인하기 위한 빈 컬럼 생성
changwon_node['inside'] = 0

#### TAAS 데이터

In [86]:
TAAS = gpd.GeoDataFrame.from_file("도시교통정보센터/TAAS.shp")
TAAS

,SAFEDATA_I,CENTER_CD,LOCATION_Y,LOCATION_D,ADDRESS_JI,LINK_ID,VIEW_YN,VIEW_CNT,REPORT_DAT,ADDRESS_NE,...,LOCATION_T,REG_DATE,ADDR_CD,LOCATION_X,LOCATION_1,DATA_DESC,f_nm,f_type,flag,geometry
0,L94100091845,L94,Y,"POLYGON((128.8079337 35.1025699,128.8076013 35...",경상남도 창원시 진해구 청안동 새안골삼거리 부근,None,Y,None,16/12/31,None,...,C0102,16/12/31,4812916200,128.806382555012,35.1012951641414,None,TAAS,POLYGON,추출가능_shp생성,"POLYGON ((128.80638 35.09949, 128.80595 35.099..."
1,L94100091850,L94,Y,"POLYGON((128.6447494 35.2515411,128.6444164 35...",경상남도 창원시 의창구 명서동(매화교 부근),None,Y,None,16/12/31,None,...,C0102,16/12/31,4812112400,128.643195449911,35.2502664293531,None,TAAS,POLYGON,추출가능_shp생성,"POLYGON ((128.64320 35.24846, 128.64277 35.248..."
2,L94100091631,L94,Y,"POLYGON((128.7035525 35.1555402,128.7032199 35...",경상남도 창원시 진해구 석동 동부초등학교 부근,None,Y,None,16/12/31,None,...,C0102,16/12/31,4812914000,128.702000366625,35.1542655078478,None,TAAS,POLYGON,추출가능_shp생성,"POLYGON ((128.70200 35.15246, 128.70157 35.152..."
3,L94101472207,L94,Y,POLYGON((128.61826117745434 35.253394754329491...,경상남도 창원시 의창구 팔용동(원풍아파트 부근),None,Y,None,19/12/19,None,...,C0102,19/12/19,4812112900,128.6160631,35.25341505,None,TAAS,POLYGON,추출가능_shp생성,"POLYGON ((128.61826 35.25339, 128.61825 35.253..."
4,L94101336341,L94,Y,"POLYGON((128.5720595 35.2051589,128.5716664 35...",경상남도 창원시 마산합포구 서성동(- 인근),None,Y,None,18/12/31,None,...,C0102,18/12/31,4812512000,128.5697299,35.20324689,None,TAAS,POLYGON,추출가능_shp생성,"POLYGON ((128.56973 35.20054, 128.56921 35.200..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,L94100096201,L94,Y,POLYGON((128.62886125102108 35.264592174376197...,서울특별시 송파구 방이동(올림픽공원남4문 부근),None,Y,None,16/12/31,None,...,C0102,16/12/31,1171011100,127.1173572,37.51586855,None,TAAS,POLYGON,추출가능_shp생성,"POLYGON ((128.62886 35.26459, 128.62885 35.264..."
335,L94100096174,L94,Y,POLYGON((128.58644490829116 35.220901904374159...,서울특별시 중랑구 면목동(기아큐서비스 부근),None,Y,None,16/12/31,None,...,C0102,16/12/31,1126010100,127.0803978,37.5928483,None,TAAS,POLYGON,추출가능_shp생성,"POLYGON ((128.58644 35.22090, 128.58643 35.220..."
336,L94100096115,L94,Y,POLYGON((128.57623178493591 35.207762587544345...,서울특별시 강동구 천호동(광진교앞 부근),None,Y,None,16/12/31,None,...,C0102,16/12/31,1174010900,127.1244158,37.54128659,None,TAAS,POLYGON,추출가능_shp생성,"POLYGON ((128.57623 35.20776, 128.57622 35.207..."
337,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [87]:
TAAS.drop([337,338], axis=0, inplace=True)

In [88]:
index_list = []
for i in tqdm(range(len(TAAS))):
    changwon_node['inside'] = changwon_node.apply(lambda x: TAAS['geometry'][i].contains(Point(x['x'], x['y'])), axis=1)
    index_list.append(list(changwon_node[changwon_node['inside'] ==True].index))
# 마지막 에러는 TAAS 데이터 마지막에 None값 때문

100%|████████████████████████████████████████████████████████████████████████████████| 337/337 [02:19<00:00,  2.41it/s]


In [89]:
index_list1 = index_list
result = sum(index_list1, [])
result1 = list(set(result))
index = result1

In [90]:
# 인덱스리스트 안의 index번호가 있는 것들을 True로 표시
changwon_node.loc[index, 'inside'] = True
changwon_node

,osmid,osmid_original,x,y,street_count,lon,lat,highway,ref,geometry,inside
0,0,"[356102207, 4665552566]",128.577201,35.220253,0,NaN,NaN,NaN,NaN,POINT (128.57720125002797 35.22025335001377),False
1,1,3866854856,128.576606,35.219793,4,128.576606,35.219793,NaN,NaN,POINT (128.5766063 35.2197931),False
2,2,2910221305,128.575747,35.221205,3,128.575747,35.221205,NaN,NaN,POINT (128.5757468 35.2212052),False
3,4005,3866854853,128.578431,35.219390,4,128.578431,35.219390,NaN,NaN,POINT (128.5784315 35.21938999999999),True
4,2471,2183525893,128.580741,35.219552,3,128.580741,35.219552,NaN,NaN,POINT (128.5807414 35.219552199999995),True
...,...,...,...,...,...,...,...,...,...,...,...
10314,10304,9740469241,128.595765,35.241808,3,128.595765,35.241808,NaN,NaN,POINT (128.5957645 35.2418085),False
10315,10305,9739424075,128.595491,35.243850,3,128.595491,35.243850,NaN,NaN,POINT (128.5954908 35.2438501),False
10316,10306,9739424082,128.596605,35.244207,3,128.596605,35.244207,NaN,NaN,POINT (128.5966053 35.2442068),False
10317,10314,9740451680,128.611620,35.244049,3,128.611620,35.244049,NaN,NaN,POINT (128.6116203 35.2440486),False


In [91]:
# 컬럼명 변경후 True 값은 1로 False 값은 0으로 변환
changwon_node.rename(columns={'inside':'TAAS'},inplace=True)
changwon_node['TAAS'].replace(False, 0, inplace=True) 
changwon_node['TAAS'].replace(True, 1, inplace=True) 

#### SAFETY_ZONE 데이터

In [93]:
sz = pd.read_excel('도시교통정보센터/보호구역_지자체.xlsx')
sz

,SNCT_SEQ,SAFEDATA_ID,LOCATION_YN,LOCATION_DATA,ADDRESS_JIBUN,LINK_ID,REPORT_DATE,ADDRESS_NEW,L_CD,M_CD,S_CD,LOCATION_TYPE_CD,ADDR_CD,LOCATION_X,LOCATION_Y,DATA_DESC,SPEED_LIMIT
0,35,NaN,Y,MULTIPOLYGON(((128.5786376753863 38.2028134592...,강원도 속초시 교동 779-16 교동초등학교,NaN,2019-03-30 15:00:00.000,강원도 속초시 만천1길 67 (교동),SF3,SF300,-,C0102,42210,128.575293,38.203225,교동초등학교,30
1,36,NaN,Y,MULTIPOLYGON(((128.5757440002959 38.2058710496...,강원도 속초시 교동 779-16 교동초등학교,NaN,2019-03-30 15:00:00.000,강원도 속초시 만천1길 67 (교동),SF3,SF300,-,C0102,42210,128.575293,38.203225,교동초등학교,30
2,37,NaN,Y,MULTIPOLYGON(((128.57615837545308 38.197233840...,강원도 속초시 교동 979 속초시 노인종합복지관,NaN,2019-03-30 15:00:00.000,강원도 속초시 수복로 42 (교동),SF3,SF301,-,C0102,42210,128.576131,38.196530,속초시노인종합복지관,30
3,38,NaN,Y,MULTIPOLYGON(((128.57078451593915 38.198705678...,강원도 속초시 노학동 1-36,NaN,2019-03-30 15:00:00.000,강원도 속초시 배움터길 42 (노학동),SF3,SF300,-,C0102,42210,128.567155,38.198883,소야초등학교,30
4,39,NaN,Y,MULTIPOLYGON(((128.57078451593915 38.198705678...,강원도 속초시 노학동 1-36,NaN,2019-03-30 15:00:00.000,강원도 속초시 배움터길 42 (노학동),SF3,SF300,-,C0102,42210,128.567155,38.198883,소야초등학교 병설유치원,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29864,49692,NaN,Y,"POLYGON((126.5525455799973 37.63861223768145,1...",경기도 김포시 대곶면 약암리 503-3,NaN,2019-03-30 15:00:00.000,경기도 김포시 대곶면 약암로948번길 10,SF3,SF301,NaN,C0102,46110,126.553804,37.638520,약암1리 경로당,30
29865,49693,NaN,Y,POLYGON((126.53598879867052 37.726055764835806...,경기도 김포시 월곶면 포내리 76-3,NaN,2022-07-22 06:43:22.435,경기도 김포시 월곶면 김포대로2970번길 39,SF3,SF301,NaN,C0102,41461,126.537943,37.726072,포내2리 마을회관,30
29866,49694,NaN,Y,"POLYGON((126.55222646516164 37.65426647333511,...",경기도 김포시 대곶면 대명리 636 대명2리마을회관,NaN,2022-07-22 06:51:35.270,경기도 김포시 대곶면 대곶로 86-17,SF3,SF301,NaN,C0102,41461,126.553138,37.652558,대명2리 마을회관,30
29867,49695,NaN,Y,POLYGON((126.54216009629481 37.677716394217825...,경기도 김포시 대곶면 쇄암리 284-8,NaN,2022-07-22 07:00:16.198,경기도 김포시 대곶면 쇄암로 128,SF3,SF301,NaN,C0102,41461,126.541767,37.678883,쇄암리 마을회관,30


In [94]:
# sz 데이터프레임의 LOCATION_DATA를 geometry로 재정의
sz['geometry'] = sz['LOCATION_DATA'].apply(wkt.loads)
gdf_sz = GeoDataFrame(sz, crs="EPSG:4326", geometry='geometry')

In [95]:
# 폴리곤 내부에 있는지 확인하기 위한 빈 컬럼 생성
changwon_node['sz'] = 0

In [96]:
# 안전구역 데이터중 창원 지역 데이터만 남김
gdf_sz = gdf_sz[gdf_sz['ADDRESS_JIBUN'].str.contains('창원', na=False)] 

In [97]:
# 인덱스 초기화
gdf_sz.reset_index(inplace=True)
gdf_sz

,index,SNCT_SEQ,SAFEDATA_ID,LOCATION_YN,LOCATION_DATA,ADDRESS_JIBUN,LINK_ID,REPORT_DATE,ADDRESS_NEW,L_CD,M_CD,S_CD,LOCATION_TYPE_CD,ADDR_CD,LOCATION_X,LOCATION_Y,DATA_DESC,SPEED_LIMIT,geometry
0,2421,4923,NaN,Y,MULTIPOLYGON(((128.6401873416394 35.2589657933...,경상남도 창원시 의창구 도계동 357-1 도계초등학교,NaN,2019-03-30 15:00:00.000,경상남도 창원시 의창구 하남천동길 10 (도계동),SF3,SF300,-,C0102,48330,128.639455,35.259333,도계초등학교,30,"MULTIPOLYGON (((128.64019 35.25897, 128.64013 ..."
1,2422,4924,NaN,Y,MULTIPOLYGON(((128.64019773805614 35.258974734...,경상남도 창원시 의창구 도계동 357-1 도계초등학교,NaN,2019-03-30 15:00:00.000,경상남도 창원시 의창구 하남천동길 10 (도계동),SF3,SF300,-,C0102,48310,128.639455,35.259333,도계초등학교,30,"MULTIPOLYGON (((128.64020 35.25897, 128.63880 ..."
2,2423,4925,NaN,Y,MULTIPOLYGON(((128.63926326802385 35.260083019...,경상남도 창원시 의창구 도계동 357-1 도계초등학교,NaN,2019-03-30 15:00:00.000,경상남도 창원시 의창구 하남천동길 10 (도계동),SF3,SF300,-,C0102,48121,128.639455,35.259333,도계초등학교,30,"MULTIPOLYGON (((128.63926 35.26008, 128.63795 ..."
3,2424,4926,NaN,Y,MULTIPOLYGON(((128.6401873416394 35.2589657933...,경상남도 창원시 의창구 도계동 357-1 도계초등학교,NaN,2019-03-30 15:00:00.000,경상남도 창원시 의창구 하남천동길 10 (도계동),SF3,SF300,-,C0102,48121,128.639455,35.259333,도계초등학교,30,"MULTIPOLYGON (((128.64019 35.25897, 128.64013 ..."
4,2425,4927,NaN,Y,MULTIPOLYGON(((128.64019773805614 35.258974734...,경상남도 창원시 의창구 도계동 357-1 도계초등학교,NaN,2019-03-30 15:00:00.000,경상남도 창원시 의창구 하남천동길 10 (도계동),SF3,SF300,-,C0102,48121,128.639455,35.259333,도계초등학교,30,"MULTIPOLYGON (((128.64020 35.25897, 128.63880 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,29492,49285,NaN,Y,"POLYGON((128.710372450816 35.14748006651033,12...",경상남도 창원시 진해구 자은동 396 자은초등학교,NaN,2021-02-05 18:51:41.103,경상남도 창원시 진해구 자은로88번길 41 (자은동),SF3,SF300,NaN,C0102,41220,128.710871,35.149101,자은초등학교,30,"POLYGON ((128.71037 35.14748, 128.71028 35.147..."
412,29561,49359,NaN,Y,POLYGON((128.66343764667224 35.224320717313915...,경상남도 창원시 성산구 중앙동 114 내동초등학교,NaN,2019-03-30 15:00:00.000,경상남도 창원시 성산구 두대로 185 (중앙동),SF3,SF300,NaN,C0102,30110,128.662135,35.225083,내동초등학교,30,"POLYGON ((128.66344 35.22432, 128.66458 35.225..."
413,29562,49360,NaN,Y,"POLYGON((128.67731200933022 35.2322064001207,1...",경상남도 창원시 의창구 용호동 68-1 용호초등학교,NaN,2019-03-30 15:00:00.000,경상남도 창원시 의창구 용지로169번길 76 (용호동),SF3,SF300,NaN,C0102,30110,128.675800,35.233777,용호초등학교,30,"POLYGON ((128.67731 35.23221, 128.67747 35.232..."
414,29587,49387,NaN,Y,"POLYGON((128.65307914859204 35.15346627190483,...",경상남도 창원시 진해구 도천동 12 도천초등학교,NaN,2019-03-30 15:00:00.000,경상남도 창원시 진해구 충장로 10 (도천동),SF3,SF300,NaN,C0102,30110,128.653216,35.152206,도천초등학교,30,"POLYGON ((128.65308 35.15347, 128.65459 35.153..."


In [98]:
# 필요없는 컬럼 삭제
gdf_sz.drop(columns=['index', 'SNCT_SEQ', 'SAFEDATA_ID', 'LOCATION_YN',
       'LINK_ID', 'REPORT_DATE', 'L_CD',
       'M_CD', 'S_CD', 'LOCATION_TYPE_CD', 'ADDR_CD'], inplace=True)

In [99]:
# GeoDataFrame을 Shapefile로 저장
gdf_sz.crs= "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
gdf_sz.to_file('safety_zone.shp', driver='ESRI Shapefile')

In [101]:
index_list = []
for i in tqdm(range(len(gdf_sz))):
    changwon_node['sz'] = changwon_node.apply(lambda x: gdf_sz['geometry'][i].contains(Point(x['x'], x['y'])), axis=1)
    index_list.append(list(changwon_node[changwon_node['sz'] ==True].index))

100%|████████████████████████████████████████████████████████████████████████████████| 416/416 [02:53<00:00,  2.39it/s]


In [102]:
index_list1 = index_list
result = sum(index_list1, [])
result1 = list(set(result))
index = result1

In [104]:
# 인덱스리스트 안의 index번호가 있는 것들을 True로 표시
changwon_node.loc[index, 'sz'] = True

In [105]:
# True 값은 1로 False 값은 0으로 변환
changwon_node['sz'].replace(False, 0, inplace=True) 
changwon_node['sz'].replace(True, 1, inplace=True) 
changwon_node

,osmid,osmid_original,x,y,street_count,lon,lat,highway,ref,geometry,TAAS,sz
0,0,"[356102207, 4665552566]",128.577201,35.220253,0,NaN,NaN,NaN,NaN,POINT (128.57720125002797 35.22025335001377),0,0
1,1,3866854856,128.576606,35.219793,4,128.576606,35.219793,NaN,NaN,POINT (128.5766063 35.2197931),0,0
2,2,2910221305,128.575747,35.221205,3,128.575747,35.221205,NaN,NaN,POINT (128.5757468 35.2212052),0,0
3,4005,3866854853,128.578431,35.219390,4,128.578431,35.219390,NaN,NaN,POINT (128.5784315 35.21938999999999),1,1
4,2471,2183525893,128.580741,35.219552,3,128.580741,35.219552,NaN,NaN,POINT (128.5807414 35.219552199999995),1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
10314,10304,9740469241,128.595765,35.241808,3,128.595765,35.241808,NaN,NaN,POINT (128.5957645 35.2418085),0,1
10315,10305,9739424075,128.595491,35.243850,3,128.595491,35.243850,NaN,NaN,POINT (128.5954908 35.2438501),0,0
10316,10306,9739424082,128.596605,35.244207,3,128.596605,35.244207,NaN,NaN,POINT (128.5966053 35.2442068),0,0
10317,10314,9740451680,128.611620,35.244049,3,128.611620,35.244049,NaN,NaN,POINT (128.6116203 35.2440486),0,0


In [107]:
changwon_node.to_csv('창원(TAAS,SAFETY_ZONE).csv')